In [ ]:
import json

import pandas as pd
import pytz

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly, plot_yearly
from fbprophet.serialize import model_to_json, model_from_json

import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

## Data Scope
- Duration from ```2017-01-01``` to ```2022-03-01```
- Searching 4 keywords ["#นัดเย็ดกทม", "#onsกทม", "#fwbกทม", "#นัดเย็ดกรุงเทพ"]

### All User Data

In [ ]:
df_user = pd.read_csv("data/total_user_data_20170101_20220301.csv") 
df_user.drop(columns = "Unnamed: 0", inplace = True)

In [ ]:
# Drop duplication on 'id'
df_user.drop_duplicates(subset = "id", inplace = True)

df_user["created_at_dt_thtz"] = pd.to_datetime(df_user["created_at_dt_thtz"])
df_user['created_at_date_thtz'] = pd.to_datetime(df_user['created_at_date_thtz'])
df_user['created_year_month'] = df_user['created_at_date_thtz'].dt.strftime('%Y-%m')

# Date Conversion
df_user['created_at_dt_month'] = df_user['created_at_dt_thtz'].dt.month
df_user['created_at_dt_month_name'] = df_user['created_at_dt_thtz'].dt.month_name()
df_user["created_at_dateweek"]= df_user['created_at_dt_thtz'].dt.weekday
df_user["created_at_dateweek_name"] = df_user['created_at_dt_thtz'].dt.day_name()
df_user["created_at_dt_hour"] = df_user["created_at_dt_thtz"].dt.hour

df_user = df_user[df_user['created_year_month'] != '2022-03']
df_user.reset_index(drop = True, inplace = True)

In [ ]:
num_by_day = df_user.groupby("created_at_date_thtz").agg({"id":"count"}).reset_index()

pio.templates.default = "simple_white"
px.line(num_by_day, x = "created_at_date_thtz", y ="id",
        title = "Number of created account : Daily All keywords",
        width = 900, height= 400)

In [ ]:
num_by_yearmonth = df_user.groupby("created_year_month").agg({"id":"count"}).reset_index()

pio.templates.default = "simple_white"
px.line(num_by_yearmonth, x = "created_year_month", y ="id",
        title = "Number of created account : Monthly All keywords",
        width = 900, height= 400)

## Tweet Data

In [ ]:
df_tweet = pd.read_csv("data/total_tweet_data_20170101_20220301.csv")

In [ ]:
'''
Data Cleaning
- Drop un-related column
- Drop duplicate Tweet id
- Convert created date to "Year-Month" for aggregation
'''

df_tweet.drop(columns = ["Unnamed: 0"], inplace = True)
df_tweet.drop_duplicates(subset = "id", inplace = True)

df_tweet['created_at_dt_thtz'] = pd.to_datetime(df_tweet['created_at_dt_thtz'])
df_tweet['created_at_date_thtz'] = pd.to_datetime(df_tweet['created_at_date_thtz'])
df_tweet['created_year_month'] = df_tweet['created_at_date_thtz'].dt.strftime('%Y-%m')

# Date Conversion
df_tweet['created_at_dt_month'] = df_tweet['created_at_dt_thtz'].dt.month
df_tweet['created_at_dt_month_name'] = df_tweet['created_at_dt_thtz'].dt.month_name()
df_tweet["created_at_dateweek"]= df_tweet['created_at_dt_thtz'].dt.weekday
df_tweet["created_at_dateweek_name"] = df_tweet['created_at_dt_thtz'].dt.day_name()
df_tweet["created_at_dt_hour"] = df_tweet["created_at_dt_thtz"].dt.hour

# Drop latest montn out due to incomplete data
df_tweet = df_tweet[df_tweet['created_year_month'] != '2022-03']
df_tweet.reset_index(drop= True, inplace = True)

### Overall Number of Tweet

In [ ]:
num_by_monthly = df_tweet.groupby(["created_year_month"]).agg({"id":"count"}).reset_index()
num_by_monthly.rename(columns = {"id" : "Number of Tweet",  
                                   "created_year_month" : "Tweet Date"}, inplace = True)

pio.templates.default = "simple_white"
px.line(num_by_monthly, x = "Tweet Date", y ="Number of Tweet",
        title = "Number of Tweet : Monthly Total",
        width = 700, height= 400)

In [ ]:
num_by_keymonthly = df_tweet.groupby(["created_year_month", "keywords"]).agg({"id":"count"}).reset_index()
num_by_keymonthly.rename(columns = {"id" : "Number of Tweet",  
                                   "keywords" : "#hashtag",
                                   "created_year_month" : "Tweet Date"}, inplace = True)

pio.templates.default = "simple_white"
num_by_keymonthly_line = px.line(num_by_keymonthly, x = "Tweet Date", y ="Number of Tweet", color = "#hashtag",
                                title = "Number of Tweet : Monthly by keywords",
                                width = 700, height= 400)
num_by_keymonthly_line
#num_by_keymonthly_line.write_html("Number of Tweet - Monthly Keywords.html")

### Tweet Frequency

In [ ]:
tweetnum_user_monthly = df_tweet.groupby(['author_id', "created_year_month"]).agg({"id" : "count"}).reset_index()
tweetnum_user_monthly.rename(columns = {"author_id" : "Account", 
                                        "id" : "Number of Tweet", 
                                        "created_year_month" : "Tweet Date"}, inplace = True)                    

In [ ]:
px.histogram(tweetnum_user_monthly[tweetnum_user_monthly['Number of Tweet'] < 50], 
            x = "Number of Tweet", title = "Count of Tweet Number Monthly Frequency/User : < 50 Tweet Month",
            width = 700, height= 500)

In [ ]:
px.histogram(tweetnum_user_monthly[tweetnum_user_monthly['Number of Tweet'] >= 50], 
            x = "Number of Tweet", title = "Count of Tweet Number Monthly Frequency/User : > 50 Tweet Month",
            width = 700, height= 500)

### Trend and Seasoanlity
**Options 1**
- Use prophet model to extract ```trend``` and ```seasoanlity of data

**Options 2**
- Count amount of account and tweet by period of Month/ Day of Week/ Time of Day

### Time-Series Model
#### Account Trends

In [ ]:
daily_user = df_user.groupby(['created_at_dt_thtz']).agg({"id" : "count"}).reset_index()
daily_user.rename(columns = {"created_at_dt_thtz" : "ds" , "id" : "y"}, inplace = True)
daily_user['ds'] = daily_user['ds'].dt.tz_localize(None)

In [ ]:
#user_model = Prophet(daily_seasonality= True, weekly_seasonality= True, yearly_seasonality= True)
#user_model.fit(daily_user)
#
#with open("./model/user_model.json", "w") as f_out:
#    json.dump(model_to_json(user_model), f_out) # Save Trained User Model

In [ ]:
with open("./model/user_model.json", "r") as f_in:
    user_model = model_from_json(json.load(f_in)) # Load Trained User Model

In [ ]:
historic_date = daily_user[['ds']]
user_forecasting = user_model.predict(historic_date)

In [ ]:
plot_components_plotly(user_model, user_forecasting, figsize = (700, 200))

#### Tweet Trends

In [ ]:
daily_tweet = df_tweet.groupby(['created_at_dt_thtz']).agg({"id" : "count"}).reset_index()
daily_tweet.rename(columns = {"created_at_dt_thtz" : "ds" , "id" : "y"}, inplace = True)
daily_tweet['ds'] = daily_tweet['ds'].dt.tz_localize(None)

In [ ]:
#tweets_model = Prophet(daily_seasonality= True, weekly_seasonality= True, yearly_seasonality= True)
#tweets_model.fit(daily_tweet)
#with open("./model/tweet_model.json", "w") as f_out:
#    json.dump(model_to_json(tweets_model), f_out) # Save Trained Tweet Model

In [ ]:

with open("./model/tweet_model.json", "r") as f_in:
    tweets_model = model_from_json(json.load(f_in)) # Load Trained Tweet Model 
    
historic_date_tweet = daily_tweet[['ds']]
tweet_forecasting = tweets_model.predict(historic_date_tweet)

In [ ]:
plot_components_plotly(tweets_model, tweet_forecasting, figsize = (700, 200))

### Count by Periods
#### Account Seasonality

In [ ]:
df_user_montly_keyword = df_user.groupby(["created_at_dt_month", "created_at_dt_month_name", "keywords"]).agg({"id":"count"}).reset_index()
df_user_montly = df_user.groupby(["created_at_dt_month", "created_at_dt_month_name"]).agg({"id":"count"}).reset_index()
df_user_montly_keyword.rename(columns = {'created_at_dt_month' : 'Month', 
                                'created_at_dt_month_name' : 'Month Name' , 
                                "keywords" : "Hashtag",
                                'id' : 'Account'} ,inplace = True)
df_user_montly.rename(columns = {'created_at_dt_month' : 'Month', 
                                'created_at_dt_month_name' : 'Month Name' , 
                                'id' : 'Account'} ,inplace = True)

df_user_weekly_keyword = df_user.groupby(["created_at_dateweek", "created_at_dateweek_name", "keywords"]).agg({"id":"count"}).reset_index()
df_user_weekly = df_user.groupby(["created_at_dateweek", "created_at_dateweek_name"]).agg({"id":"count"}).reset_index()
df_user_weekly_keyword.rename(columns = {'created_at_dateweek' : 'Day of Week', 
                                'created_at_dateweek_name' : 'Day of Week Name' , 
                                "keywords" : "Hashtag",
                                'id' : 'Account'} ,inplace = True)
df_user_weekly.rename(columns = {'created_at_dateweek' : 'Day of Week', 
                                'created_at_dateweek_name' : 'Day of Week Name' ,
                                'id' : 'Account'} ,inplace = True)


df_user_hourly_keyword = df_user.groupby(["created_at_dt_hour", "keywords"]).agg({"id":"count"}).reset_index()
df_user_hourly = df_user.groupby(["created_at_dt_hour"]).agg({"id":"count"}).reset_index()
df_user_hourly_keyword.rename(columns = {'created_at_dt_hour' : 'Hour', 
                                "keywords" : "Hashtag",
                                'id' : 'Account'} ,inplace = True)
df_user_hourly.rename(columns = {'created_at_dt_hour' : 'Hour', 
                                'id' : 'Account'} ,inplace = True)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(x=df_user_montly['Month'],
                            y=df_user_montly['Account'],
                            mode='lines', name = "Monthly Seasonality"), row=1, col=1)

fig.append_trace(go.Scatter(x=df_user_weekly['Day of Week'],
                            y=df_user_weekly['Account'],
                            mode = 'lines', name = "Day of Week Seasonality"), row=2, col=1)

fig.append_trace(go.Scatter(x=df_user_hourly['Hour'],
                            y=df_user_hourly['Account'],
                            mode = 'lines', name = "Hour Seasonality"), row=3, col=1)

fig.update_layout(height=600, width=700, title_text="Seasonality of Account Creation")
fig.show()

In [ ]:
fig = make_subplots(rows=4, cols=1)
fig.append_trace(go.Scatter(x=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "onsกทม"]['Month'],
                            y=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "onsกทม"]['Account'],
                            mode='lines', name = "onsกทม"), row=1, col=1)

fig.append_trace(go.Scatter(x=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "นัดเย็ดกทม"]['Month'],
                            y=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "นัดเย็ดกทม"]['Account'],
                            mode = 'lines', name = "นัดเย็ดกทม"), row=2, col=1)

fig.append_trace(go.Scatter(x=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "นัดเย็ดกรุงเทพ"]['Month'],
                            y=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "นัดเย็ดกรุงเทพ"]['Account'],
                            mode = 'lines', name = "นัดเย็ดกรุงเทพ"), row=3, col=1)

fig.append_trace(go.Scatter(x=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "fwbกทม"]['Month'],
                            y=df_user_montly_keyword[df_user_montly_keyword['Hashtag'] == "fwbกทม"]['Account'],
                            mode = 'lines', name = "fwbกทม"), row=4, col=1)

fig.update_layout(height=600, width=700, title_text="Seasonality of Account by Monthly : by Keywords")
fig.show()

#### Tweets Seasonality

In [ ]:
df_tweet_montly_keyword = df_tweet.groupby(["created_at_dt_month", "created_at_dt_month_name", "keywords"]).agg({"id":"count"}).reset_index()
df_tweet_montly = df_tweet.groupby(["created_at_dt_month", "created_at_dt_month_name"]).agg({"id":"count"}).reset_index()
df_tweet_montly_keyword.rename(columns = {'created_at_dt_month' : 'Month', 
                                'created_at_dt_month_name' : 'Month Name' , 
                                "keywords" : "Hashtag",
                                'id' : 'Account'} ,inplace = True)
df_tweet_montly.rename(columns = {'created_at_dt_month' : 'Month', 
                                'created_at_dt_month_name' : 'Month Name' , 
                                'id' : 'Account'} ,inplace = True)

df_tweet_weekly_keyword = df_tweet.groupby(["created_at_dateweek", "created_at_dateweek_name", "keywords"]).agg({"id":"count"}).reset_index()
df_tweet_weekly = df_tweet.groupby(["created_at_dateweek", "created_at_dateweek_name"]).agg({"id":"count"}).reset_index()
df_tweet_weekly_keyword.rename(columns = {'created_at_dateweek' : 'Day of Week', 
                                'created_at_dateweek_name' : 'Day of Week Name' , 
                                "keywords" : "Hashtag",
                                'id' : 'Account'} ,inplace = True)
df_tweet_weekly.rename(columns = {'created_at_dateweek' : 'Day of Week', 
                                'created_at_dateweek_name' : 'Day of Week Name' ,
                                'id' : 'Account'} ,inplace = True)


df_tweet_hourly_keyword = df_tweet.groupby(["created_at_dt_hour", "keywords"]).agg({"id":"count"}).reset_index()
df_tweet_hourly = df_tweet.groupby(["created_at_dt_hour"]).agg({"id":"count"}).reset_index()
df_tweet_hourly_keyword.rename(columns = {'created_at_dt_hour' : 'Hour', 
                                "keywords" : "Hashtag",
                                'id' : 'Account'} ,inplace = True)
df_tweet_hourly.rename(columns = {'created_at_dt_hour' : 'Hour', 
                                'id' : 'Account'} ,inplace = True)

In [ ]:
fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(x=df_tweet_montly['Month'],
                            y=df_tweet_montly['Account'],
                            mode='lines', name = "Monthly Seasonality"), row=1, col=1)

fig.append_trace(go.Scatter(x=df_tweet_weekly['Day of Week'],
                            y=df_tweet_weekly['Account'],
                            mode = 'lines', name = "Day of Week Seasonality"), row=2, col=1)

fig.append_trace(go.Scatter(x=df_tweet_hourly['Hour'],
                            y=df_tweet_hourly['Account'],
                            mode = 'lines', name = "Hour Seasonality"), row=3, col=1)

fig.update_layout(height=600, width=700, title_text="Seasonality of Tweet Posted")
fig.show()

### Location Analysis

In [ ]:
df_geononna = df_tweet[~df_tweet['geo'].isna()]
df_geona = df_tweet[df_tweet['geo'].isna()]

In [ ]:
df_geononna['geo_dict'] = df_geononna['geo'].apply(lambda x: json.loads(x.replace("'", "\"")))
df_geononna['place_id'] = df_geononna['geo_dict'].apply(lambda x:  x['place_id'] if 'place_id' in x else None)
df_geononna['coordinates'] = df_geononna['geo_dict'].apply(lambda x:  x['coordinates']['coordinates'] if 'coordinates' in x else None)

In [ ]:


for text_body in df_tweet['text']:
    if "จุฬา" in text_body:
        print(text_body)
        print("----")
        print("")